In [24]:
fastfood_list = [
    {"Product": "Coke", "Price": 1.99},
    {"Product": "Fries", "Price": 2.49},  # "Pommes" translated to "Fries"
    {"Product": "Hamburger", "Price": 3.99},
    {"Product": "Cheeseburger", "Price": 4.49},
    {"Product": "Chicken Nuggets", "Price": 4.99},
    {"Product": "Ice Cream Sundae", "Price": 2.99},  # "Eisbecher" translated to "Ice Cream Sundae"
    {"Product": "Salad", "Price": 3.99},  # "Salat" translated to "Salad"
    {"Product": "Fishburger", "Price": 4.99},
    {"Product": "Hot Dog", "Price": 3.49},
    {"Product": "Milkshake", "Price": 2.99}  # "Milchshake" translated to "Milkshake"
]


In [25]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

audio_file = open("speech.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

In [26]:
def find_products_by_name(produktliste, namen):
    gefundene_produkte = [produkt for produkt in produktliste if produkt["Product"] in namen]
    return gefundene_produkte


gesuchte_namen = ["Coke", "Hamburger"]
gefundene_produkte = find_products_by_name(fastfood_list, gesuchte_namen)

In [27]:
for produkt in gefundene_produkte:
    print(produkt)

{'Product': 'Coke', 'Price': 1.99}
{'Product': 'Hamburger', 'Price': 3.99}


In [12]:
tools = [{
    "type": "function",
    "function": {
        "name": "find_products_by_name",
        "description": "Find products by their names from a list of product dictionaries",
        "parameters": {
            "type": "object",
            "properties": {
                "product_list": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "Product": {"type": "string"},
                            "Price": {"type": "number"}
                        },
                        "required": ["Product", "Price"]
                    },
                    "description": "A list of product dictionaries, each containing a product's name and price"
                },
                "names": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of product names to search for in the product list"
                }
            },
            "required": ["product_list", "names"]
        }
    }
}]


In [13]:
import json
from openai import OpenAI

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

In [16]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I want to order 1 coke and one Hot Dog!"}
  ],
  tools=tools
)

print(completion.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PwdUm2cvBL3Oepe0RjvrqRei', function=Function(arguments='{"product_list":[{"Product":"Coke","Price (€)":1.5},{"Product":"Hot Dog","Price (€)":3}],"names":["Coke","Hot Dog"]}', name='find_products_by_name'), type='function')])


In [21]:
message = completion.choices[0].message
message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PwdUm2cvBL3Oepe0RjvrqRei', function=Function(arguments='{"product_list":[{"Product":"Coke","Price (€)":1.5},{"Product":"Hot Dog","Price (€)":3}],"names":["Coke","Hot Dog"]}', name='find_products_by_name'), type='function')])

In [23]:
print(message.tool_calls[0].function.name)
print(message.tool_calls[0].function.arguments)

find_products_by_name
{"product_list":[{"Product":"Coke","Price (€)":1.5},{"Product":"Hot Dog","Price (€)":3}],"names":["Coke","Hot Dog"]}
